# Coordinates and World Coordinate Systems

This notebook will give you a introduction to solar physics coordinate systems and the tools availble in Python for coverting between pixel coordinates and different physical coordinate systems. 

<section class="objectives panel panel-warning">
<div class="panel-heading">
<h3><span class="fa fa-certificate"></span> Learning Objectives </h3>
</div>
<ul>
    <li>Projected Coordinate Systems in Images</li>
    <li>World Coordinate Systems</li>
    <li>Using WCS to calculate coordinates in images</li>
</ul>
</section>

## Projected Coordinate Systems

<div style="float:left; width:39%">
<p>
When taking images of the sky, we are projecting the spherical celestial coordinate system onto a 2-dimensional plane, which means that there is no simple linear relation between pixel coordinates and celestial coordinates
</p>
<p>
There are multiple coordinate systems used to describe the locations in 2D and 3D space for both Astronomy and Solar Physics. We shall use a couple of these systems here as examples but if you want to know more about them there are many of resources avalible.
</p>
<p>
Useful coordinate and WCS resources:
<ol>
<li>
<a href="http://www.aanda.org/10.1051/0004-6361:20054262">
Thompson, W. T. Coordinate systems for solar image data. Astronomy and Astrophysics 449, 791–803 (2006).
</a>
</li>
<li>
<a href="http://www.edpsciences.org/10.1051/0004-6361:20021326">
Greisen, E. W. & Calabretta, M. R. Representations of world coordinates in FITS. Astronomy and Astrophysics 395, 1061–1075 (2002).
</a>
</li>
<li>
<a href="http://www.edpsciences.org/10.1051/0004-6361:20021327">
Calabretta, M. R. & Greisen, E. W. Representations of celestial coordinates in FITS. Astronomy and Astrophysics 395, 1077–1122 (2002).
</a>
</li>
<li>
<a href="https://fits.gsfc.nasa.gov/fits_wcs.html">
FITS WCS Page
</a>
</li>
</ol>

</div>
<div style="float:left; width:59%">
<img src="coord_inset.png" width=100% />
</div>

### Physical Coordinate Systems

Projected coordinate systems are one type of physical coordinate systems, they are the one we will be focusing on in this lesson due to their applicability to imaging data. Astropy and SunPy support representing point in many different physical coordinate systems, both projected and fully 3D, such as ICRS or Helioprojective.

SunPy supports the following coordinate systems from Thompson (2006):

* Helioprojective (Cartesian)
* Heliocentric
* Heliographic (Stonyhurst)
* Heliographic (Carrington)

In [3]:
import sunpy.coordinates
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
hpc = SkyCoord(100*u.arcsec, 700*u.arcsec, frame='helioprojective')
hpc

In [ ]:
hpc.transform_to('heliographic_stonyhurst')

## World Coordinate Systems

### From pixels  to physical coordinates

The FITS files have a standard for describing the physical coordinate system associated with imaging data, this is called the world coordinate system or WCS, sometimes the specific FITS version of this is referred to as FITS-WCS.

There are multiple papers describing the FITS-WCS standard for various types of data, there is a list here: http://fits.gsfc.nasa.gov/fits_wcs.html

As you learned in the previous lesson we can load FITS files with Astropy. To demonstrate a simple example of a FITS file with FITS-WCS information in the header we shall use an image from SunPy:

In [4]:
from sunpy.data.sample import AIA_171_IMAGE
from astropy.io import fits

hdulist = fits.open(AIA_171_IMAGE)
hdulist.verify('silentfix')
hdulist[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   32 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 1024 / length of data axis 1                          
NAXIS2  =                 1024 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
ORIGIN  = 'SDO/JSOC-SDP'                                                        
DATE    = '2011-03-19T11:08:25'                                                 
TELESCOP= 'SDO/AIA'                                                             
INSTRUME= 'AIA_3'                                                               
DATE-OBS= '2011-03-19T10:54:

As you can see there are lots of keys in this and most other real world FITS headers. The ones we need to understand for FITS-WCS are:

Reference Pixel and Coordinate:

In [ ]:
header = hdulist[0].header

print(header['CRVAL1'], header['CRVAL2'])
print(header['CRPIX1'], header['CRPIX2'])

Pixel resolution (at the reference pixel):

In [ ]:
print(header['CDELT1'], header['CDELT2'])

Rotation angle, in degress (at the reference pixel):

In [ ]:
print(header['CROTA2'])

Coordinate System and Projection:

In [ ]:
print(header['CTYPE1'], header['CTYPE2'])

We could now sit down and work out how to convert from a pixel coordinate to a physical coordinate described by this header (Helioprojective).

However, we can cheat and just use Astropy.

In [10]:
from astropy.wcs import WCS

wcs = WCS(header)

We can convert from pixel to world coordinate:

In [ ]:
wcs.wcs_pix2world(((100, 100),), 0)

Or back again:

In [ ]:
wcs.wcs_world2pix([[  3.59725669e+02,  -2.74328093e-01]], 0)

The last parameter to the two above examples is the 'origin' parameter. It is a flag that tells WCS if you indexes should be 0-based (like numpy) or 1-based (like FITS).
Here we are using 0 as we want to convert to and from numpy indexes of the array.

To make these numbers more in a form you are familiar with, we can use Astropy's Longitude and Latitude classes:

In [ ]:
from astropy.coordinates import Longitude, Latitude

In [ ]:
lon, lat = wcs.wcs_pix2world(((100, 100),), 0)[0]
lon, lat

In [ ]:
lon = Longitude(lon*u.deg, wrap_angle=180*u.deg).to(u.arcsec)
lat = Latitude(lat*u.deg).to(u.arcsec)

lon, lat

<section class="objectives panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> How large is the image? </h2>
</div>
<br/>
To get a little practise using Astropy's WCS calculate the world coordinates of the following pixels:
<code>
[-500, 0]
[500, 500]
[0, 0]
</code>
<br/>
</section>

In [ ]:
print(wcs.wcs_pix2world(((-500, 0),), 1))
print(wcs.wcs_pix2world(((500, 500),), 1))
print(wcs.wcs_pix2world(((0, 0),), 1))